In [20]:
import pandas as pd
import numpy as np
import keras
import glob
import os
import random
import pickle
from skimage import io
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, InputLayer
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import ZeroPadding2D, Dropout
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [21]:
test = Image.open(r"C:\Users\mitch\OneDrive\Documents\Metis DL\corn_data\Healthy\Corn_Health (1).jpg")
size = (128, 128)

In [22]:
base_path = "C:\\Users\\mitch\\OneDrive\\Documents\\Metis DL\\corn_data\\"
healthy_path = base_path+'Healthy\\'
Blight_path = base_path+'Blight\\'
Rust_path = base_path+'Common_Rust\\'
GraySpot_path = base_path+'Gray_Leaf_Spot\\'

In [23]:
Healthy_pixelArray = []
Healthy_labelArray = []
Blight_pixelArray = []
Blight_labelArray = []
Rust_pixelArray = []
Rust_labelArray = []
GraySpot_pixelArray = []
GraySpot_labelArray = []

In [24]:
for i in range(len(os.listdir(healthy_path))):
    currentImage = Image.open(healthy_path+os.listdir(healthy_path)[i])
#     resizedPic = tf.image.resize(currentImage, size)
    resizedPic = tf.image.resize_with_pad(currentImage, 128, 128)
    ravelledPic = np.ravel(resizedPic)
    Healthy_pixelArray.append(ravelledPic)
    Healthy_labelArray.append('Healthy')

In [25]:
for i in range(len(os.listdir(Blight_path))):
    currentImage = Image.open(Blight_path+os.listdir(Blight_path)[i])
#     resizedPic = tf.image.resize(currentImage, size)
    resizedPic = tf.image.resize_with_pad(currentImage, 128, 128)
    ravelledPic = np.ravel(resizedPic)
    Blight_pixelArray.append(ravelledPic)
    Blight_labelArray.append('Blight')

In [26]:
for i in range(len(os.listdir(Rust_path))):
    currentImage = Image.open(Rust_path+os.listdir(Rust_path)[i])
#     resizedPic = tf.image.resize(currentImage, size)
    resizedPic = tf.image.resize_with_pad(currentImage, 128, 128)
    ravelledPic = np.ravel(resizedPic)
    Rust_pixelArray.append(ravelledPic)
    Rust_labelArray.append('Rust')

In [27]:
for i in range(len(os.listdir(GraySpot_path))):
    currentImage = Image.open(GraySpot_path+os.listdir(GraySpot_path)[i])
#     resizedPic = tf.image.resize(currentImage, size)
    resizedPic = tf.image.resize_with_pad(currentImage, 128, 128)
    ravelledPic = np.ravel(resizedPic)
    GraySpot_pixelArray.append(ravelledPic)
    GraySpot_labelArray.append('GraySpot')

In [28]:
mega_pixelArray = Healthy_pixelArray+Blight_pixelArray+Rust_pixelArray+GraySpot_pixelArray
mega_labelArray = Healthy_labelArray+Blight_labelArray+Rust_labelArray+GraySpot_labelArray

In [29]:
pixel_df = pd.DataFrame(mega_pixelArray)
label_df = pd.DataFrame(mega_labelArray)

In [30]:
label_df['Healthy'] = label_df[0].apply(lambda x: 1 if x == 'Healthy' else 0)
label_df['Blight'] = label_df[0].apply(lambda x: 1 if x == 'Blight' else 0)
label_df['Rust'] = label_df[0].apply(lambda x: 1 if x == 'Rust' else 0)
label_df['GraySpot'] = label_df[0].apply(lambda x: 1 if x == 'GraySpot' else 0)

In [32]:
label_df.head(20)

,0,Healthy,Blight,Rust,GraySpot
0,Healthy,1,0,0,0
1,Healthy,1,0,0,0
2,Healthy,1,0,0,0
3,Healthy,1,0,0,0
4,Healthy,1,0,0,0
5,Healthy,1,0,0,0
6,Healthy,1,0,0,0
7,Healthy,1,0,0,0
8,Healthy,1,0,0,0
9,Healthy,1,0,0,0


In [33]:
label_df = label_df.drop(0, axis=1)
labelArray1Hot = np.array(label_df)
drop_list = range(49152, 65536)
pixel_df.drop(columns = drop_list, inplace = True)
pixelArray = pixel_df.values.reshape((-1,128,128,3)) #49152

In [34]:
labelArray1Hot

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]], dtype=int64)

In [19]:
leafs = ['<b>Healthy</b>', '<b>Blight</b>', '<b>Rust</b>', '<b>Grey Spot</b>']
sets = [1162, 1146, 1306, 574]
color_map=['#71A92C', '#ffffbf', '#c8592c', 'D3D3D3']

fig = go.Figure(data=[go.Pie(labels=leafs, values=sets, pull=[0.1, 0, 0, 0])])
fig.update_traces(hole=.4, textposition='inside', textinfo='percent+label',
                 marker=dict(colors=color_map, line=dict(color='#000000', width=2)))
fig.show()

# pull=[0, 0, 0, 0, 0, 0.1]
# font = 20, 

In [ ]:
X_train, X_test, y_train, y_test = (train_test_split(pixelArray, labelArray1Hot, 
                                                     test_size = .2, random_state = 420))

with open('X_train', 'wb') as f:
    pickle.dump(X_train,f)
with open('X_test', 'wb') as f:
    pickle.dump(X_test,f)
with open('y_train', 'wb') as f:
    pickle.dump(y_train,f)
with open('y_test', 'wb') as f:
    pickle.dump(y_test,f)